In [22]:
import json
import pickle
import random
from collections import Counter
import os
import copy
from Vocab import Vocab

file_group = 'validation'    #availabe:  train    test    validation    tune

with open('vocab.pk', 'rb') as f:
    vocab = pickle.load(f)

In [11]:
# pseudo labels for split
with open(''.join(['split_data_set/', file_group, '_complex_sents.pk']), 'rb') as f:
    complex_sents = pickle.load(f)
pseudo_labels=copy.deepcopy(complex_sents)
with open(''.join(['split_data_set/', file_group, '_complex_sent_lens.pk']), 'rb') as f:
    complex_sent_lens = pickle.load(f)
    
# generate pseudo labels by inserting three tokens: '.', '<split>' and 'eos'
split_range=0
for idx, sent in enumerate(pseudo_labels):
    split_idx = int(complex_sent_lens[idx]/2)
    split_idx = random.randint(split_idx-split_range, split_idx+split_range)
    sent.insert(split_idx, vocab.word2token['.'])
    sent.insert(split_idx+1, vocab.word2token['<split>'])
    sent.insert(complex_sent_lens[idx]+2, vocab.word2token['<eos>'])
    sent_len = len(sent)
    for _ in range(61-sent_len):
        sent.append(vocab.word2token['<padding>'])

In [12]:
rand_idx = random.randint(0, len(complex_sents)-1)
print(complex_sents[rand_idx], len(complex_sents[idx]))
print(pseudo_labels[rand_idx], len(pseudo_labels[idx]))

[28541, 3990, 15364, 9194, 789, 23563, 25, 27945, 789, 27513, 21467, 38225, 11673, 17835, 21354, 21467, 18327, 43713, 19863, 33147, 12697, 40533, 23661, 25168, 434, 33942, 27513, 21467, 12697, 3990, 25377, 40017, 4786, 11883, 37734, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 50
[28541, 3990, 15364, 9194, 789, 23563, 25, 27945, 789, 27513, 21467, 38225, 11673, 17835, 21354, 21467, 18327, 37734, 5, 43713, 19863, 33147, 12697, 40533, 23661, 25168, 434, 33942, 27513, 21467, 12697, 3990, 25377, 40017, 4786, 11883, 37734, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 61


In [13]:
with open(''.join(['split_data_set/', file_group, '_pseudo_labels.pk']), 'wb') as f:
    pickle.dump(pseudo_labels, f)

In [23]:
# pseudo labels for fusion
with open(''.join(['split_data_set/', file_group, '_complex_sents.pk']), 'rb') as f:
    complex_sents = pickle.load(f)
pseudo_simple_sents=copy.deepcopy(complex_sents)
pseudo_labels=copy.deepcopy(complex_sents)
with open(''.join(['split_data_set/', file_group, '_complex_sent_lens.pk']), 'rb') as f:
    complex_sent_lens = pickle.load(f)
pseudo_simple_sent_lens = copy.deepcopy(complex_sent_lens)

# generate pseudo simple sentences by inserting two tokens: '.', '<split>'
split_range=0
for idx, sent in enumerate(pseudo_simple_sents):
    split_idx = int(pseudo_simple_sent_lens[idx]/2)
    split_idx = random.randint(split_idx-split_range, split_idx+split_range)
    sent.insert(split_idx, vocab.word2token['.'])
    sent.insert(split_idx+1, vocab.word2token['<split>'])
    
    pseudo_simple_sent_lens[idx] = pseudo_simple_sent_lens[idx]+2
    
for idx, sent in enumerate(pseudo_labels):
    if sent[-1]!=vocab.word2token['<padding>']:
        sent.append(vocab.word2token['<eos>'])
    else:
        for ii in range(49, -1, -1):
            if sent[ii]!=vocab.word2token['<padding>']:
                sent.insert(ii+1, vocab.word2token['<eos>'])
                break

In [25]:
rand_idx = random.randint(0, len(complex_sents)-1)
print(complex_sents[rand_idx], len(complex_sents[rand_idx]), complex_sent_lens[rand_idx])

print(pseudo_simple_sents[rand_idx], pseudo_simple_sent_lens[rand_idx])
print(pseudo_labels[rand_idx], len(pseudo_labels[rand_idx]))

[3, 25168, 30341, 29740, 13264, 31006, 13264, 32498, 21467, 21092, 18637, 27513, 22111, 13264, 28248, 21467, 26817, 789, 3, 789, 13126, 27567, 34917, 14895, 37734, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 50 25
[3, 25168, 30341, 29740, 13264, 31006, 13264, 32498, 21467, 21092, 18637, 27513, 37734, 5, 22111, 13264, 28248, 21467, 26817, 789, 3, 789, 13126, 27567, 34917, 14895, 37734, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 27
[3, 25168, 30341, 29740, 13264, 31006, 13264, 32498, 21467, 21092, 18637, 27513, 22111, 13264, 28248, 21467, 26817, 789, 3, 789, 13126, 27567, 34917, 14895, 37734, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 51


In [26]:
with open(''.join(['fusion_data_set/', file_group, '_pseudo_simple_sent_lens.pk']), 'wb') as f:
    pickle.dump(pseudo_simple_sent_lens, f)
with open(''.join(['fusion_data_set/', file_group, '_pseudo_simple_sents.pk']), 'wb') as f:
    pickle.dump(pseudo_simple_sents, f)
with open(''.join(['fusion_data_set/', file_group, '_pseudo_labels.pk']), 'wb') as f:
    pickle.dump(pseudo_labels, f)

In [5]:
# # pseudo labels for fusion
# with open(''.join(['fusion_data_set/', file_group, '_simple_sents.pk']), 'rb') as f:
#     simple_sents = pickle.load(f)

# with open(''.join(['fusion_data_set/', file_group, '_simple_sent_lens.pk']), 'rb') as f:
#     simple_sent_lens = pickle.load(f)
    

In [6]:
pseudo_labels=copy.deepcopy(simple_sents)

In [7]:
# # generate pseudo labels by removing two tokens: '.', '<split>' and inserting three tokens:'eos'
# for sent_idx, sent in enumerate(pseudo_labels):
#     for idx, token in enumerate(sent):
#         if token == vocab.word2token['<split>']:
#             split_idx = idx
#             break
#     sent.pop(split_idx-1)
#     sent.pop(split_idx-1)
#     sent.insert(simple_sent_lens[sent_idx]-1-2+1, vocab.word2token['<eos>'])
#     if len(sent)!=59:
#         print(sent_idx, len(sent))
#         break

In [8]:
# rand_idx = random.randint(0, len(simple_sents)-1)
# print(simple_sents[rand_idx], len(simple_sents[idx]))
# print(pseudo_labels[rand_idx], len(pseudo_labels[idx]))

[21467, 25133, 24152, 36047, 11687, 40850, 11138, 3, 5327, 35165, 11138, 3, 5327, 25161, 13264, 11732, 27513, 11382, 13264, 752, 32201, 3990, 36519, 24152, 36047, 37734, 5, 28541, 7447, 9194, 11206, 39575, 29243, 36047, 13264, 25546, 43775, 32514, 25168, 11951, 32514, 40017, 21467, 1400, 25168, 23842, 30705, 21354, 21467, 25133, 24152, 21354, 32201, 11687, 8558, 25161, 37734, 1, 1, 1] 60
[21467, 25133, 24152, 36047, 11687, 40850, 11138, 3, 5327, 35165, 11138, 3, 5327, 25161, 13264, 11732, 27513, 11382, 13264, 752, 32201, 3990, 36519, 24152, 36047, 28541, 7447, 9194, 11206, 39575, 29243, 36047, 13264, 25546, 43775, 32514, 25168, 11951, 32514, 40017, 21467, 1400, 25168, 23842, 30705, 21354, 21467, 25133, 24152, 21354, 32201, 11687, 8558, 25161, 37734, 2, 1, 1, 1] 59


In [9]:
with open(''.join(['fusion_data_set/', file_group, '_pseudo_labels.pk']), 'wb') as f:
    pickle.dump(pseudo_labels, f)